In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
with open ("./feats/all_spliced.txt") as feature_file:
    train_features = feature_file.readlines()

In [ ]:
with open("./alis/gzs/ali.post2") as train_labels:
    train_labels = train_labels.readlines()

In [ ]:
final_labels = []
use = "train"
for line in train_labels:
    labels_list = line.strip().split("[")
    labels_list = [re.sub("\]", "", ele.strip()).strip().split()[0] for ele in labels_list]
    speaker_id, sentence_id = labels_list[0].split("_")
    alis_post = np.array([int(ele) for ele in labels_list[1:]])
    frame_number = 1
    for label in alis_post:
        final_labels.append([speaker_id, sentence_id, frame_number, use, label])
        frame_number += 1

In [ ]:
# labels_final = []
# for i in final_labels:
#     for j in i:
#         labels_final.append(int(j))

In [ ]:
frame_number = 1
use = "train"
dummy_np = []
for line in train_features:
    line = re.sub(" \[", "", line.strip())
    line = re.sub(" \]", "", line.strip())
    if("_" in line):
        speaker_id, sentence_id = line.split("_")
        frame_number = 1
    else:
        mfcc = [float(ele) for ele in line.split(" ")]
        dummy_np.append([speaker_id, sentence_id, frame_number, use, mfcc, "None"])
        frame_number += 1

In [ ]:
feature_df = pd.DataFrame(columns=["speaker_id", "sentence_id", "frame_number", "use", "feature_vector", "label"])

In [ ]:
speaker_ids = [ele[0] for ele in dummy_np]
sentence_ids = [ele[1] for ele in dummy_np]
frame_numbers = [ele[2] for ele in dummy_np]
use = [ele[3] for ele in dummy_np]
features = [ele[4] for ele in dummy_np]
label = [ele[5] for ele in dummy_np]

In [ ]:
feature_df["speaker_id"] = speaker_ids
feature_df["sentence_id"] = sentence_ids 
feature_df["frame_number"] = frame_numbers
feature_df["use"] = use 
feature_df["feature_vector"] = features
feature_df["label"] = label

In [ ]:
label_df = pd.DataFrame(columns=["speaker_id", "sentence_id", "frame_number", "use", "label"])

In [ ]:
final_labels[0]

In [ ]:
speaker_ids = [ele[0] for ele in final_labels]
sentence_ids = [ele[1] for ele in final_labels]
frame_numbers = [ele[2] for ele in final_labels]
use = [ele[3] for ele in final_labels]
label = [ele[4] for ele in final_labels]

In [ ]:
label_df["speaker_id"] = speaker_ids
label_df["sentence_id"] = sentence_ids 
label_df["frame_number"] = frame_numbers
label_df["use"] = use 
label_df["label"] = label

In [ ]:
feature_df.to_pickle("raw_feats.pkl")
label_df.to_pickle("raw_labels.pkl")

In [ ]:
new_df = pd.merge(feature_df, label_df,  how='inner', left_on=['speaker_id','sentence_id', 'frame_number'], right_on = ['speaker_id','sentence_id', 'frame_number'])

In [ ]:
new_df = new_df.drop("label_x", 1)

In [ ]:
new_df = new_df.drop("use_x", 1)

In [ ]:
for col in new_df:
    print(col)

In [ ]:
new_df.columns = ["speaker_id", "sentence_id", "frame_number", "feature_vector", "use", "label"]

In [ ]:
len(new_df["label"].tolist())

In [ ]:
def append_features(Xtrain):
    new_Xtrain = []
    zero_array = np.zeros((1,40))[0]
    for i in range(len(Xtrain)):
        if(i==0):
            new_Xtrain.append(np.hstack((zero_array,zero_array,zero_array,zero_array,zero_array,Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],Xtrain[i+5])))
        elif(i==1):
            new_Xtrain.append(np.hstack((zero_array,zero_array,zero_array,zero_array,Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],Xtrain[i+5])))
        elif(i==2):
            new_Xtrain.append(np.hstack((zero_array,zero_array,zero_array,Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],Xtrain[i+5])))
        elif(i==3):
            new_Xtrain.append(np.hstack((zero_array,zero_array,Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],Xtrain[i+5])))
        elif(i==4):
            new_Xtrain.append(np.hstack((zero_array,Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],Xtrain[i+5])))

        elif(i==len(Xtrain)-5):
            new_Xtrain.append(np.hstack((Xtrain[i-5],Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],zero_array)))
        elif(i==len(Xtrain)-4):
            new_Xtrain.append(np.hstack((Xtrain[i-5],Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],zero_array,zero_array)))
        elif(i==len(Xtrain)-3):
            new_Xtrain.append(np.hstack((Xtrain[i-5],Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],zero_array,zero_array,zero_array)))
        elif(i==len(Xtrain)-2):
            new_Xtrain.append(np.hstack((Xtrain[i-5],Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],zero_array,zero_array,zero_array,zero_array)))
        elif(i==len(Xtrain)-1):
            new_Xtrain.append(np.hstack((Xtrain[i-5],Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],zero_array,zero_array,zero_array,zero_array,zero_array)))

            
        else:
            new_Xtrain.append(np.hstack((Xtrain[i-5],Xtrain[i-4],Xtrain[i-3],Xtrain[i-2],Xtrain[i-1],Xtrain[i],Xtrain[i+1],Xtrain[i+2],Xtrain[i+3],Xtrain[i+4],Xtrain[i+5])))
        if(i%100000 == 0):
            print(i)
    return(new_Xtrain)

In [ ]:
# new_df["feature_vector"] = append_features(new_df["feature_vector"].tolist())

In [ ]:
len(new_df["feature_vector"].tolist()[0])

In [ ]:
len(new_df["feature_vector"][0])

In [ ]:
new_df["label"].tolist()[100000]

In [ ]:
new_df.to_hdf("kaldi_spliced_440_post_new.hd5", key="timit")

In [ ]:
new_df.memory_usage(deep=True).sum()